In [ ]:
# ===================================================
# 🧭 Notebook 6 — Interactive Dashboard
# ===================================================
# Purpose:
# Move from static model interpretation (Notebook 5)
# → to *interactive exploration* and *real-time insights*.
#
# In this notebook we will:
# ---------------------------------------------------
# 1️⃣ Load trained models and feature data.
# 2️⃣ Build an interactive Streamlit or Dash dashboard.
# 3️⃣ Allow filtering and comparison of runs.
# 4️⃣ Enable on-demand predictions and SHAP explanations.
# 5️⃣ Visualize performance trends and model reasoning.
# ---------------------------------------------------
#
# 💡 The goal: Turn your ML pipeline into a transparent,
# exploratory interface where you can *see* and *understand*
# what drives performance, fatigue, and training outcomes.
# ===================================================


# ---------------------------------------------------
# 📦 1. Setup & Imports
# ---------------------------------------------------
import pandas as pd
import numpy as np
import joblib
import shap
import matplotlib.pyplot as plt
import streamlit as st   # or dash if preferred
from pathlib import Path

# ---------------------------------------------------
# 📁 2. Load Models and Data
# ---------------------------------------------------
# Paths
data_path = Path("data/processed/runs_summary.csv")
model_path = Path("models/random_forest_clf.pkl")
explainer_path = Path("models/shap_explainer_clf.pkl")

# Load
summary_df = pd.read_csv(data_path, parse_dates=["date"])
rf_clf = joblib.load(model_path)
explainer_clf = joblib.load(explainer_path)

available_features = [
    "total_distance_km", "duration_min", "avg_cadence",
    "total_elev_gain", "load_7d", "load_28d",
    "fastest_1km_pace", "fastest_5min_pace",
    "pace_variability", "avg_gct_est_ms", "cadence_drift"
]

# ---------------------------------------------------
# 🎛️ 3. Streamlit App Structure (example layout)
# ---------------------------------------------------
st.title("🏃‍♂️ Running Insights Dashboard")
st.markdown("""
Interactively explore your model predictions and SHAP explanations.
Use the sidebar to select runs, view predictions, and understand why
the model made its choice.
""")

# Sidebar selection
selected_date = st.sidebar.selectbox(
    "Select a run date", summary_df["date"].sort_values(ascending=False)
)
case = summary_df[summary_df["date"] == selected_date]
case_X = case[available_features].iloc[[0]]

# Prediction & SHAP
pred_label = rf_clf.predict(case_X)[0]
shap_vec, _ = _shap_vector_for_sample(explainer_clf, rf_clf, case_X)
contrib = pd.Series(shap_vec, index=case_X.columns).sort_values(
    key=lambda x: x.abs(), ascending=False
)

st.subheader(f"🏷️ Predicted Cluster: **{pred_label}**")
st.bar_chart(contrib.head(15))

# ---------------------------------------------------
# 📊 4. Future Extensions
# ---------------------------------------------------
# - Add global SHAP summary and trend plots.
# - Add run-comparison mode (select two runs and compare).
# - Integrate live Garmin/Strava API data.
# - Add metrics: fatigue index, rolling performance load, etc.
# - Export visualizations to HTML or PDF.
# ---------------------------------------------------
